# Контекст

- задача резюмирования текста
- модель T5-large

In [1]:
import torch
import json
import warnings
from transformers import T5Tokenizer,\
                         T5ForConditionalGeneration,\
                         T5Config

import pandas as pd




warnings.filterwarnings("ignore")
print('CUDA ?: ', torch.cuda.is_available())
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

CUDA ?:  True


# Скачивание модели T5

In [2]:
model = T5ForConditionalGeneration.from_pretrained('t5-large')
tokenizer = T5Tokenizer.from_pretrained('t5-large')

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
print(model.config)

T5Config {
  "_name_or_path": "t5-large",
  "architectures": [
    "T5ForConditionalGeneration"
  ],
  "classifier_dropout": 0.0,
  "d_ff": 4096,
  "d_kv": 64,
  "d_model": 1024,
  "decoder_start_token_id": 0,
  "dense_act_fn": "relu",
  "dropout_rate": 0.1,
  "eos_token_id": 1,
  "feed_forward_proj": "relu",
  "initializer_factor": 1.0,
  "is_encoder_decoder": true,
  "is_gated_act": false,
  "layer_norm_epsilon": 1e-06,
  "model_type": "t5",
  "n_positions": 512,
  "num_decoder_layers": 24,
  "num_heads": 16,
  "num_layers": 24,
  "output_past": true,
  "pad_token_id": 0,
  "relative_attention_max_distance": 128,
  "relative_attention_num_buckets": 32,
  "task_specific_params": {
    "summarization": {
      "early_stopping": true,
      "length_penalty": 2.0,
      "max_length": 200,
      "min_length": 30,
      "no_repeat_ngram_size": 3,
      "num_beams": 4,
      "prefix": "summarize: "
    },
    "translation_en_to_de": {
      "early_stopping": true,
      "max_length": 300,
 

In [8]:
model = model.to(device)

def summarize(text, max_len):
    preprocess_text = text.strip().replace('\n', '')
    t5_prepare = 'summarize: ' + preprocess_text

    tokenized_text = tokenizer.encode(t5_prepare, return_tensors = 'pt').to(device)
    summary_ids = model.generate(
        inputs = tokenized_text,
        num_beams = 4,
        no_repeat_ngram_size = 2,
        min_length = 30,
        max_length = max_len,
        early_stopping = True
    )

    output = tokenizer.decode(
        summary_ids[0],
        skip_special_tokens = True
    )

    return output

In [14]:
text ="""
The United States Declaration of Independence was the first Etext
released by Project Gutenberg, early in 1971. The title was stored
in an emailed instruction set which required a tape or diskpack be
hand mounted for retrieval. The diskpack was the size of a large
cake in a cake carrier, cost $1500, and contained 5 megabytes, of
which this file took 1-2%. Two tape backups were kept plus one on
paper tape. The 10,000 files we hope to have online by the end of
2001 should take about 1-2% of a comparably priced drive in 2001.
"""



summary = summarize(text, 100)

In [15]:
import pprint

pprint.pprint(summary, width = 50)

('the united states declaration of independence '
 'was the first etext published by project '
 'gutenberg, early in 1971. the 10,000 files we '
 'hope to have online by the end of2001 should '
 'take about 1-2% of a comparably priced drive '
 'in 2001.')
